# Preprocessing

In [1]:
import datasets # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import es_core_news_sm # Modelo Spacy de procesamiento de lenguaje natural en español
import spacy # Biblioteca de procesamiento de lenguaje natural
import pandas as pd # Biblioteca de manejo de conjuntos de datos
import re # Módulo de expresiones regulares
import tokenizers # Biblioteca de tokenización de texto
import nltk # Biblioteca de procesamiento de lenguaje natural

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


## 🤗 Datasets

🤗 (HuggingFace) Datasets es una biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural que se destaca por la simplicidad de sus métodos y el gran repositorio 🤗 Hub que contiene muchos conjuntos de datos libres para descargar sólo con una linea de Python.

En nuestro curso trabajaremos con `spanish_diagnostics`, un conjunto de datos de nuestro grupo investigación PLN@CMM que contiene textos de sospechas diagnósticas de la lista de espera chilena y está etiquetado con el destino de la interconsulta; este destino puede ser `dental` o `no_dental`.

In [2]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Con esta linea descargamos el conjunto de datos completo

Using custom data configuration default
Reusing dataset spanish_diagnostics (C:\Users\ville\.cache\huggingface\datasets\spanish_diagnostics\default\0.0.0\45c176cea64580ea9631f78c2867a657ede368597681e5337e9f1c976e4e84ff)


Nuestro conjunto de datos cuenta con 2 particiones, una partición `train` y otra `test`.

In [3]:
spanish_diagnostics

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 30000
    })
})

En esta clase utilizaremos la partición `train` del conjunto de datos.

In [4]:
spanish_diagnostics["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 70000
})

Podemos acceder facilmente a atributos de nuestro `Dataset`.

- `shape`: Tal como en muchas otras bibliotecas de python este atributo contiene la forma de nuestro conjunto de datos con la sintaxis `(filas, columnas)`.
- `column_names`: Este atributo contiene el nombre de las características que tiene nuestro conjunto de datos. En nuestro caso tenemos una característica `text`, la cual contiene la hipótesis diagnóstica del conjunto de datos y `label` que contiene el destino al cual fue referido.
- `features`: Este atributo nos describe la clase a la que pertenece cada una de las características. En nuestro caso `text` es un `string` y `label` es del tipo `ClassLabel` con 2 clases con nombre `not_dental` y `dental`.

In [5]:
spanish_diagnostics["train"].shape

(70000, 2)

In [6]:
spanish_diagnostics["train"].column_names

['label', 'text']

In [7]:
spanish_diagnostics["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_dental', 'dental'], names_file=None, id=None)}

Tal como en muchas otras clases de datos en Python podemos acceder a subconjuntos de datos a través de sus índices.

In [8]:
spanish_diagnostics["train"][0]

{'label': 1,
 'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR'}

In [9]:
spanish_diagnostics["train"][:3]

{'label': [1, 0, 0],
 'text': ['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
  'OBTRUCCION FOSA NASAL DERECHA',
  'Perturbación de la actividad y de la atención Trastorno defícit atencional']}

In [10]:
spanish_diagnostics["train"][1,3,5]

{'label': [0, 0, 1],
 'text': ['OBTRUCCION FOSA NASAL DERECHA',
  'M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA',
  'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria']}

También podemos acceder a cada una de las características por separado.

In [11]:
spanish_diagnostics["train"]['text'][:3]

['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'OBTRUCCION FOSA NASAL DERECHA',
 'Perturbación de la actividad y de la atención Trastorno defícit atencional']

Con 🤗 Datasets también podemos trabajar en otras bibliotecas, como por ejemplo importar el conjunto de datos en Pandas.

In [12]:
spanish_diagnostics_train_df = pd.DataFrame(spanish_diagnostics["train"])

In [13]:
spanish_diagnostics_train_df

,label,text
0,1,- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUS...
1,0,OBTRUCCION FOSA NASAL DERECHA
2,0,Perturbación de la actividad y de la atención ...
3,0,M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 A...
4,1,PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A C...
...,...,...
69995,1,DM1 Evaluación
69996,1,ABCESO SUBMUCOSO PIEZA 2.6
69997,0,Pbs Inmunodeficiencia
69998,0,"QUISTE SINOVIAL DEL HUECO POPLITEO, DE BAKER"


Verificamos que nuestro conjunto de datos tiene sus clases balanceadas.

In [14]:
spanish_diagnostics_train_df.label.value_counts()

1    35034
0    34966
Name: label, dtype: int64

Si tenemos localmente un conjunto de datos y queremos importarlo a 🤗 Datasets también podemos hacerlo. Aquí importamos el conjunto de datos desde un archivo CSV.

In [15]:
datasets.load_dataset('csv', data_files='spanish_diagnostics/spanish_diagnostics.csv')

Using custom data configuration default-68d4ea5b370697bf
Reusing dataset csv (C:\Users\ville\.cache\huggingface\datasets\csv\default-68d4ea5b370697bf\0.0.0\2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


DatasetDict({
    train: Dataset({
        features: ['diagnostic', 'is_dental'],
        num_rows: 100000
    })
})

## Normalización

Una de las tareas que podemos realizar sobre las características no estructuradas de texto es la normalización. La cual consiste en llevar nuestro texto a una forma más consistente a lo largo del conjunto de datos.

Podemos observar que nuestro conjunto de datos cuenta con una alta inconsistencia respecto al uso de mayúsculas, el uso de tildes y el uso de signos de puntuación.

In [16]:
spanish_diagnostics["train"]["text"][:10]

['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'OBTRUCCION FOSA NASAL DERECHA',
 'Perturbación de la actividad y de la atención Trastorno defícit atencional',
 'M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA',
 'PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A CAMARA PULPAR, EVALUAR POR ESPECIALIDAD',
 'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria',
 'PZ. 12 TREPANADA',
 'CARCINOMA TORIODEO',
 'DISPEPSIA Y METEORISMO',
 'ASA 1 DENTICION TEMPORAL MORDIDA CRUZADA']

Para poder llevar todo a minúsculas, simplemente podemos utilizar el método str.lower().

In [17]:
sample_sentence_lower = spanish_diagnostics["train"]["text"][0].lower()
sample_sentence_lower

'- anomalías dentofaciales (incluso la maloclusión)\n\n\n discrepancia dentomaxilar'

Para eliminar todo los caracteres no alfabéticos podemos utilizar un patrón de expresión regular con la sintaxis: `[^a-zñáéíóú]`, la cual se explica como:

- `^`: Este es un `NO` lógico que invierte todo lo que viene a su derecha.
- `a-z`: Este patrón coincide todos los caracteres de la `a` a la `z` (minúsculas)
- `áéíóú`: Este patrón coincide con todas las vocales con tilde.

Todos estos patrones están concatenados con un `O` lógico.

In [18]:
sample_sentence_lower_alpha = re.sub(r'[^a-zñáéíóú]', ' ', sample_sentence_lower)
sample_sentence_lower_alpha

'  anomalías dentofaciales  incluso la maloclusión     discrepancia dentomaxilar'

Reemplazamos todas las vocales con tilde con con su forma sin tilde.

In [19]:
re.sub('ó', 'o', sample_sentence_lower_alpha)

'  anomalías dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'

Agrupamos todo en una función que normalizará una cadena de texto que le pasemos.

In [20]:
def normalize(text, remove_tildes = True):
    """Normaliza una cadena de texto convirtiéndo todo a minúsculas, quitando los caracteres no alfabéticos y los tildes"""
    text = text.lower() # Llevamos todo a minúscula
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) # Reemplazamos los caracteres no alfabéticos por un espacio
    if remove_tildes:
        text = re.sub('á', 'a', text) # Reemplazamos los tildes
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

Los objetos del tipo Dataset implementan un método Dataset.map() con el cual podemo aplicar uan función a cada una de las instancias de nuesto conjunto de datos. Lo interesante de este método es que aplica la función de manera paralela.

In [21]:
spanish_diagnostics_normalized = spanish_diagnostics["train"].map(
    lambda x: { # Utilizamos una función anónima que devuelve un diccionario
        "normalized_text" : normalize(x["text"]) # Esta es una nueva característica que agregaremos a nuestro conjunto de datos.
    })

Ahora nuestro conjunto de datos cuenta con una nueva característica `normalized_text`.

In [22]:
spanish_diagnostics_normalized[0]

{'label': 1,
 'normalized_text': '  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar',
 'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR'}

## Tokenización

La tokenización es el proceso de demarcación de secciones de una cadena de caracteres. Estas secciones podrían ser oraciones, palabras o subpalabras.

El método más simple para tokenizar una cadena de caracteres en nuestro lenguaje es la separación por espacios. Aplicamos una separación por espacios mediante el método `str.split()` sobre nuestro conjunto de datos normalizado.

In [23]:
spanish_diagnostics_normalized[0]["normalized_text"]

'  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'

In [24]:
spanish_diagnostics_normalized[0]["normalized_text"].split()

['anomalias',
 'dentofaciales',
 'incluso',
 'la',
 'maloclusion',
 'discrepancia',
 'dentomaxilar']

Si bien el método de separación por espacios funciona bien en nuestro conjunto de datos normalizado, también quisiéramos tokenizar nuestro texto sin normalizar.

In [25]:
spanish_diagnostics_normalized[0]["text"]

'- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR'

In [26]:
spanish_diagnostics_normalized[0]["text"].split()

['-',
 'ANOMALÍAS',
 'DENTOFACIALES',
 '(INCLUSO',
 'LA',
 'MALOCLUSIÓN)',
 'DISCREPANCIA',
 'DENTOMAXILAR']

Al aplicar el mismo métodos podemos observar que no funciona totalmente bien debido a la presencia de caracteres no alfabéticos. Para solucionar esto, existen métodos basados en una serie de reglas para solucionar estos problemas. Utilizaremos la implementación de un tokenizador basado en reglas de la biblioteca de procesamiento de lenguaje natural Spacy.

In [27]:
spacy_tokenizer = es_core_news_sm.load().tokenizer

In [28]:
list(spacy_tokenizer(spanish_diagnostics_normalized[0]["text"]))

[-,
 ANOMALÍAS,
 DENTOFACIALES,
 (,
 INCLUSO,
 LA,
 MALOCLUSIÓN,
 ),
 
 
 
  ,
 DISCREPANCIA,
 DENTOMAXILAR]

Al utilizar el tokenizador basado en reglas, podemos tener resultados mucho mejores que los anteriores.

### 🤗 Tokenizers

🤗 también cuenta con una biblioteca llamada Tokenizers, con la cual podemos construir nuestro tokenizador basado en nuestro conjunto de datos.

Instanciamos el tokenizador con un modelo WordPiece, el cual parte construyendo un vocabulario que incluye todas los caracteres presentes en el conjunto de datos y posteriormente comienza a mezclar caracteres hasta encontrar conjuntos de caracteres que tienen más probabilidad de aparecer juntos que separados.

In [29]:
tokenizer = tokenizers.Tokenizer(tokenizers.models.WordPiece())

Esta biblioteca nos permite añadir pasos de normalización directamente. Replicamos lo mismo que hacemos con nuestra función `normalizer()`.

In [30]:
normalizer = tokenizers.normalizers.Sequence([
    tokenizers.normalizers.Lowercase(), # Llevamos todo a minúscula
    tokenizers.normalizers.NFD(), # Separamos cada caracter según los elementos que lo componen: á -> (a, ´)
    tokenizers.normalizers.StripAccents(), # Eliminamos todos los acentos
    tokenizers.normalizers.Replace(tokenizers.Regex(r"[^a-z ]"), " ") # Reemplazamos todos los caracteres no alfabéticos
])

In [31]:
normalizer.normalize_str(spanish_diagnostics_normalized[0]["text"])

'  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'

Añadimos el normalizador al tokenizador

In [32]:
tokenizer.normalizer = normalizer

Pre tokenizamos nuestro conjunto de datos mediante espacio para delimitar el tamaño que puede tener cada token.

In [33]:
tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()

Instanciamos el entrenador que entrenará nuestro tokenizador.

In [34]:
trainer = tokenizers.trainers.WordPieceTrainer()

Entrenamos el tokenizador sobre nuestro conjunto de datos.

In [35]:
tokenizer.train_from_iterator(spanish_diagnostics_normalized["text"], trainer)

Mediante el método `Tokenizer.encode()` obtenemos la representación tokenizada de nuesto texto. Esta representación contiene varios atributos, donde los más interesantes son:

- `ids`: Contiene nuestro texto representado a través de una lista que contiene los identificadores de cada token.
- `tokens`: Contiene nuestro texto representado a través de una lista que contiene el texto de cada token.

In [36]:
spanish_diagnostics_normalized[0]["text"]

'- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR'

In [37]:
tokenized_output = tokenizer.encode(spanish_diagnostics_normalized[0]["text"])

In [38]:
tokenized_output.ids

[383, 618, 635, 119, 466, 1765, 970]

In [39]:
tokenized_output.tokens

['anomalias',
 'dentofaciales',
 'incluso',
 'la',
 'maloclusion',
 'discrepancia',
 'dentomaxilar']

Tal como lo hicimos anteriormente podemos aplicar paralelamente nuestro tokenizador sobre el conjunto de datos mediante el método `Dataset.map()`

In [40]:
spanish_diagnostics_normalized_tokenized = spanish_diagnostics_normalized.map(lambda x: {"tokenized_text":tokenizer.encode(x["text"]).tokens})

Nuestro conjunto de datos ahora contiene el texto tokenizado en la característica `tokenized_text`.

In [41]:
spanish_diagnostics_normalized_tokenized[0]

{'label': 1,
 'normalized_text': '  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar',
 'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'tokenized_text': ['anomalias',
  'dentofaciales',
  'incluso',
  'la',
  'maloclusion',
  'discrepancia',
  'dentomaxilar']}

## Stemming y Lematización

Con el fin de disminuir la cantidad de características de las representaciones de texto existen métodos que reducen el tamaño de vocabulario al eliminar inflexiones que puedan tener las palabras. Estos métodos son:

- Lematización: Este método lleva una palabra en su forma flexionada a su forma base, por ejemplo *tratada* -> *tratar*
- Stemming: Este método trunca las palabras de entrada mediante un algoritmo predefinido para encontrar la raíz de la misma, por ejemplo *tratada* -> *trat*

El proceso de lematización lo haremos a través de la biblioteca Spacy y el proceso de stemming a través de la biblioteca NLTK utilizando el algoritmo Snowball.

Instanciamos el analizador de Spacy

In [42]:
nlp = es_core_news_sm.load()

Definimos como tokenizador el que entrenamos anteriormente.

In [43]:
def custom_tokenizer(text):
    tokens = tokenizer.encode(text).tokens
    return spacy.tokens.Doc(nlp.vocab,tokens)

In [44]:
nlp.tokenizer = custom_tokenizer

Instanciamos el Stemmer

In [45]:
stemmer = nltk.stem.SnowballStemmer("spanish")

Podemos verificar cómo funcionan estos métodos sobre un texto de prueba de nuestro conjunto de datos.

In [46]:
for t in nlp(spanish_diagnostics_normalized_tokenized[5]["text"]):
    print(f"Token: {t.text}\nLema: {t.lemma_}\nStem: {stemmer.stem(t.text)}\n---")

Token: pieza
Lema: pieza
Stem: piez
---
Token: n
Lema: n
Stem: n
---
Token: tratada
Lema: tratar
Stem: trat
---
Token: endodonticamente
Lema: endodonticamente
Stem: endodont
---
Token: restaurada
Lema: restaurar
Stem: restaur
---
Token: con
Lema: con
Stem: con
---
Token: ionomero
Lema: ionomero
Stem: ionomer
---
Token: y
Lema: y
Stem: y
---
Token: resina
Lema: resinar
Stem: resin
---
Token: compuesta
Lema: componer
Stem: compuest
---
Token: necesita
Lema: necesitar
Stem: necesit
---
Token: protesis
Lema: protesis
Stem: protesis
---
Token: fija
Lema: fijo
Stem: fij
---
Token: por
Lema: por
Stem: por
---
Token: gran
Lema: gran
Stem: gran
---
Token: pnrdida
Lema: pnrdida
Stem: pnrdid
---
Token: coronaria
Lema: coronario
Stem: coronari
---
